In [24]:
from google.colab import drive
#drive.mount("/content/drive/MyDrive/Microproyecto2", force_remount=True)
drive.mount("/content/drive", force_remount=True)

#tf.keras.utils.get_file(origin="https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/download", fname='covid')

Mounted at /content/drive


In [21]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib
import os
import PIL
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

 metti dataset in cartella con notebook e copia percorso

In [22]:
path_dataset_full="./Microproyecto2/Dataset"
path_test_data="./Microproyecto2/Dataset/test"
path_train_data="./Microproyecto2/Dataset/train"

In [17]:
import pathlib
path = pathlib.Path()
print(path)

.


In [18]:
data_dir_train = pathlib.Path(path_train_data)  #saving directories
print((data_dir_train))
data_dir_test = pathlib.Path(path_test_data)
print(type(data_dir_test))
print(data_dir_train.glob)

Microproyecto2/Dataset/train
<class 'pathlib.PosixPath'>
<bound method Path.glob of PosixPath('Microproyecto2/Dataset/train')>


In [19]:
image_train_count = len(list(data_dir_train.glob('*/*.jpg')))  #image count
print("cardinality of train set=",image_train_count)
image_test_count = len(list(data_dir_test.glob('*/*.jpg')))
print("cardinality of test set=",image_test_count)
#print("total images=cardinality of the dataset=",image_train_count+image_test_count)

cardinality of train set= 0
cardinality of test set= 0


In [ ]:
COVID19 = list(data_dir_train.glob('COVID19/*'))  #make a list with images whose name contains "COVID19"
PNEUMONIA = list(data_dir_train.glob('PNEUMONIA/*'))
NORMAL = list(data_dir_train.glob('NORMAL/*'))

In [ ]:
'''PIL.Image.open(str(COVID19[0]))#change index for a different image
PIL.Image.open(str(PNEUMONIA[0]))
PIL.Image.open(str(NORMAL[0]))'''

In [ ]:
batch_size = 32
img_height = 300 #original size 1800
img_width = 300 #1800

 dati di training

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

 dati di test

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_test,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

prestazioni

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
#test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalizzazione parametri

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

# Rete Neurale

In [ ]:
num_classes = 3

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()



# training